# Visualize and annotate DESeq2 results

## Review our pipeline
In the pipeline image, the red text indicates the names of the R packages we will use to perform the analysis, the yellow boxes indicate the data (raw or processed), and the grey indicates what we will do in each step. Note that we will use [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html) to identify differentially expressed genes (DEGs). This is a highly cited package that uses a negative binomial distribution to calculate DEGs. However, there are multiple pipelines and programs to do this. [Knoweng](http://education.knoweng.org/sequenceng/) has a nice interface to explore other options, including the strengths and weaknesses of each.
![](https://github.com/davidnboone/test-image/blob/master/overview-image.jpg?raw=true "pipeline overview")

## To this point we have:
1. Extracted and calculated HER2 and TNBC status of each patient.
2. Extracted count and abundance data for each transcript from those samples.
3. Prepped the transcipt count and abundance data.
4. Collapsed transcript counts to the gene level.
5. Calculated differentially expressed genes using DESeq2

## In this notebook we will:
1. Annotate the results table to include GeneSymbol and information about abundance
2. Visualize the results with a few QC plots
3. Generate a heatmap of filtered DEGs

___
___
___

## We still need the proper R packages loaded. We will need to start by reloading them with the same code.
Of course you can use the typical library() command to load each as well.




In [ ]:
#you can very easily use "install.packages" or "biocLite" to install the packages and "library" to load them
#however, instead I am installing a package called pacman that will determine if a package is already installed
#if it is not it will install it and after will load it
#this method is convenient when sharing code that requires the use of others packages

if (!require("pacman")) install.packages("pacman")
pacman::p_load(R.utils, data.table, tximport, DESeq2, biomaRt, jsonlite, BiocParallel, ggplot2, gplots, RColorBrewer, devtools, pheatmap)


## Set the working directory to the workspace set up in the previous notebook.
 You can check to deterimine if you are in the correct directory by examining your working directory. If it is not the proper workspace then change to proper directory.


In [ ]:
getwd()

In [ ]:
#only need to run if you are not in the proper working directory.
base_dir <- "/Volumes/Oesterreich"
setwd(base_dir)

project_name <- "TCGA_Her2_TNBC_DEGs"
date <- "2019_02_06" #use the same date as notebook 1,2, and 3
setwd(project_name)

outdir <- "DESeq_output" #to save all of the images and final tables


## Load the necessary files.
If you compiled all the Notebooks into one and did not restart the kernel since completing DeSEQ, you can skip this step.

For visualization and annotation we will need the following:
1. dds ("her2.tnbc.results.Rda")
2. results files (dataframe - "her2.tnbc.results.csv" and results Rda "her2.tnbc.results.Rda"))
3. gene level TPM abundance file ("brca.untran.gene.tpm.Rda")
4. final TNBC and HER2 IDs


You can load/read these using the code below. **Be sure the commands point to the appropriate directories that contain the data**


In [ ]:
#read in dds file after running DESeq. Because it was saved as an Rda we can use the 
#load() function and it will retain the appropriate variable name
load(file = file.path(date, "her2.tnbc.results.Rda"))

#read in the ordered results csv to the same variable name as when it was created.
#note we specify that the first column is the names of the rows and not a data column
results.ordered <- read.csv(file = file.path(date, outdir, "her2.tnbc.results.csv"), row.names = 1)

#load results file as Rda (before made a dataframe, which is used for visualization)
load(file = file.path(date, outdir, "her2.tnbc.results.Rda"))

#read in the abundance data to append to results file
load(file = file.path(date, "brca.untran.gene.tpm.Rda"))

#load final HER2 and TNBC IDs
load(file = file.path(date, "her2.ids.final.Rda"))
load(file = file.path(date, "tnbc.ids.final.Rda"))



___
___
___

# 1. Finalize results output table
We need to add information to the DESeq output to help visualize, explore, and export the data. In this section we will:
- a. Shrink the log fold change using a statistical function built into DESeq2.
- b. Add Gene Symbol and Biotype to the results.
- c. Add information about abundance to the results.

## 1a. Log fold change shrinkage for visualization and ranking
Before exploratory visualization of the results, it is proper to use a statistical analysis to shrink the log fold change (LFC) to account for expression. As specified in the [manual](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#the-deseqdataset), "shrinkage of effect size (LFC estimates) is useful for visualization and ranking of genes. To shrink the LFC, we pass the dds object to the function lfcShrink. Below we specify to use the apeglm method for effect size shrinkage (Zhu, Ibrahim, and Love 2018), which improves on the previous estimator." This is particularly useful to eliminate artifically inflated fold change values from genes expressed at a low level. For example, a gene may have only 1 count in HER2 but 100 in TNBC. The fold change would be 100 for this gene, but given that even 100 counts is low, this gene may be below the limits of detection and hence the differences are by chance only. 

In [ ]:
#LFC for shrinkage and visualization 
#this line will tell us the name of the coeficients we can shrink
resultsNames(dds)


In [ ]:
#LFC for shrinkage and visualization
#shrink based on the TNBC vs HER2  coeficient
#we found the name of the coeficient in the previous output
#type is the statistical test to use. There are multiple ones you can select.
results_LFC <- lfcShrink(dds, coef="BC_subtype_TNBC_vs_HER2", type="apeglm")
results_LFC.df <- as.data.frame(results_LFC)

#save after running to save computational time later
save(results_LFC.df, file = file.path(date, outdir, "results_LFC.Rda"))

#write to table so it can be easily read outside of R
write.csv(results_LFC.df, file = file.path(date, outdir, "results_LFC.df.csv"))

Let's see what the differences look like of the most significant genes

In [ ]:
results_LFC_ordered <- results_LFC.df[order(results_LFC.df$pvalue, na.last = TRUE),]

message("LFC results")
head(results_LFC_ordered)

message("raw fold change")
head(results.ordered)

## 1b. Use of BiomaRt to convert Ensembl gene Ids to Gene symbol
We can use the bioconducter package [biomaRt](https://bioconductor.org/packages/release/bioc/html/biomaRt.html) to convert Ensembl IDs to Gene Symbol, which is the typical format for readable gene names. I highly recommend reading the manual as this package can do many things not discussed in this exercise. 



In [ ]:
########################################################################################
######################## make gene names readable (gene symbol) ########################
####################### done through the use of the biomaRt package ####################
########################################################################################


#set ensembl as dataset from ensembl v81 (gencode v23). same as we used above in Tximport
#dataset referst to species
#biomart is the specific annotation
#version is the version

ensembl <- useDataset(dataset = "hsapiens_gene_ensembl", mart = useEnsembl(biomart = 'ENSEMBL_MART_ENSEMBL', version = 81))

#create list of gene ids to convert
geneidlist <- rownames(results_LFC.df)

#create data frame with gene id, gene name, gene source, and gene biotype
#attributes are what we are searching for. here we want the external gene name and i included gene biotype because it might be interesting to look at coding vs noncoding genes
#filters are what we use as input for the query
#values are what we want to translate
#mart defines what species and verion to use.
ensembl.translate <- getBM(attributes = c("ensembl_gene_id", "external_gene_name", "gene_biotype"), filters = "ensembl_gene_id", values = geneidlist, mart = ensembl)
row.names(ensembl.translate) <- ensembl.translate$ensembl_gene_id

#examine ensembl.translate
head(ensembl.translate)





Now we need to merge this data with our results file and export the intermediate table as a .csv to be easily shared with others.

In [ ]:
#merge results w/ ensembl.translate
#note row names of results.fdr.threshold are Ensembl geneids
results.merged <- merge(results_LFC.df, ensembl.translate, by.x = "row.names", by.y = "ensembl_gene_id", all.x = TRUE)


#set row.names column as actual row names
row.names(results.merged) <- results.merged$Row.names

#remove 1st column b/c repetitive
results.merged <- results.merged[, -1]

#order by padj value
results.merged.padj <- results.merged[order(results.merged$padj), ]

#see the differences between the unmerged and merged data tables
#biomart translate table
message("Ensembl Translate")
head(ensembl.translate)

#raw output table
message("Unmerged output")
head(results_LFC.df)

#merged final table
message("Final merged table")
head(results.merged.padj)

#save final merged table
write.csv(results.merged.padj, file = file.path(date, outdir, "gene_symbol_lfc_results.csv"))



## 1c. Add abundance information to results file.
In our example we arbitrarily filtered out most genes expressed at low levels before we calculated the differentially expressed genes. If you remember, we did this just to decrease the computational time and power needed to perform the analysis. Typically this would not be done. Instead, we often need to include some information about expression in the results table to allow filtering based on expression. A typical cutoff is requiring 10% of samples to be greater than 1 TPM (something that is fairly easy to confirm in the lab). But, an investigator could be more stringent and require the median or a minimum of 1 TPM. Accordingly, we will add the 90th percentile and median of both HER2 and TNBC samples to the data table. 


In [ ]:
#This block of code uses the gene-level tpm data (brca.untran.gene.tpm) 
#and the final her2 (her2.ids.final) and tnbc (tnbc.ids.final)ids
#that were calculated in Notebook 3. They were reloaded at the beginning of this notebook
#in case you started fresh

#subset tpm data to just genes in the results file and just the her2 or tnbc samples
her2.tpm.df <- brca.untran.gene.tpm[rownames(results.merged.padj), her2.ids.final]
tnbc.tpm.df <- brca.untran.gene.tpm[rownames(results.merged.padj), tnbc.ids.final]

#confirm that her2.tpm.df and tnbc.tpm.df has the same rows in same order as results.merged.padj
message("her2.tpm.df data")
nrow(her2.tpm.df)
head(her2.tpm.df)

message("tnbc.tpm.df data")
nrow(tnbc.tpm.df)
head(tnbc.tpm.df)

message("results.merged.padj data")
nrow(results.merged.padj)
head(results.merged.padj)




Because the rows of each are exactly the same we can easily calculate the median and 90th percentile in the HER2 and TNBC specific data frames and add as a column to our results file using the apply function.

In [ ]:
#use the apply function to calculate the median and make a new column in the deseq results file
results.merged.padj$median_her2 <- apply(her2.tpm.df, 1, median)
results.merged.padj$median_tnbc <- apply(tnbc.tpm.df, 1, median)

#calculate 90th percentile using quantile function
results.merged.padj$ninetieth_percentile_her2 <- apply(her2.tpm.df, 1, function(x) quantile(x, 0.9))
results.merged.padj$ninetieth_percentile_tnbc <- apply(tnbc.tpm.df, 1, function(x) quantile(x, 0.9))

                                                   
#examine the results
head(results.merged.padj)
                                                   
#save the final datatable for export
write.csv(results.merged.padj, file = file.path(date, outdir, "FINAL_annotated_TNBC_HER2_DEG_table.csv"))
                                                   

___
___
___

# 2. Visualize the results with a few quality control plots

## 2a. If you know anything about the dataseq, a quick first step to check the analysis is to confirm what you know.
In this exercise, we know that HER2+ samples should have higher HER2+ (aka ERBB2) expression than TNBC samples. You can find the ensembl ID for HER2 in many different databases including [genecards.org](https://www.genecards.org/cgi-bin/carddisp.pl?gene=ERBB2&keywords=erbb2) to subset the results file based on row, or subset based on the gene symbol name that we added above. The column name is "external_gene_name."

In [ ]:
#sanity check to make sure that ERBB2 (HER2) is differentially expressed
results.merged.padj[results.merged.padj$external_gene_name == "ERBB2", ]


### Visualize HER2 results.
From the above output you can see that HER2 is very significantly different between the two groups with an FDR (p-adj) of e-146. You can also see that the tpm data makes sense as the median of her2 is nearly 10 fold higher than the  median of tnbc. You can visualize this result as well. You could make a dotplot with ggplot using the tpm data, but to demonstrate some of the quick built in functions of DESeq2, we will make a dotplot using the built-in plotCounts function with the dds (immediate output of DESeq) as the input. This creats a dotplot of the count data. This is handy if you want to visualize before you add the abundance data.

In [ ]:
#Make a dotplot of HER2 expression

#print to screen
plotCounts(dds, gene="ENSG00000141736", intgroup = "BC_subtype")
dev.off()

#save graph as tiff to file
tiff(file.path(date, outdir, "HER2_counts_graph.tiff"))
plotCounts(dds, gene="ENSG00000141736", intgroup = "BC_subtype")
dev.off()


The dotplot confirms the analysis for HER2 (ERBB2), but other plots are more useful to visualize all the genes at once.

## 2b. Generate MA Plots
MA plots are a great first visualization of the results because they show on a single graph all genes, their raw expression, LFC, and significance (colored based on a threshold). These are the three features that are typically used when filtering DEGs for downstream analysis. **note** the plotMA function works with the raw output of the results() function in DESeq2 and not on the data tables that were created for export. Hence the use of results_LFC below instead of results.merged.padj. Again this is convenient if you want to visualize QC plots before taking the time to fully annotate the results datatable.

In [ ]:
#note this function only works on the output of the results() function in DESeq2 and not on
#the dataframe created of the results

#MA Plot of corrected LFCs
#save to file
tiff(file.path(date, outdir, "MAplot_correctedLFC.tiff"))
plotMA(results_LFC, alpha = 0.001, xlim = c(1, 10000)) #we have an idea of a limit based on the summary performed in Notebook 3
dev.off()

#print to screen
message("corrected LFCs")
plotMA(results_LFC, alpha = 0.001, xlim=c(1,10000)) #we have an idea of a limit based on the summary performed in Notebook 3
dev.off()


### Can you tell me why there are no genes with counts under 200?

## 2c. Generate Volcano plot
A volcano plot is another nice general visualization of your overall results. This graph plots the -log10 padj value as they y-axis and adjusted LFC as the x-axis.
**note** this is not a built in function of DESeq2, hence we are using the final results data table to generate the volcano plot

In [ ]:
#volcano plot
#graphics.off()
alpha <- 0.001 # Threshold on the adjusted p-value
cols <- densCols(results.merged.padj$log2FoldChange, -log10(results.merged.padj$padj))

tiff(file.path(date, outdir, "volcano.tiff"), res = 100)
plot(results.merged.padj$log2FoldChange, -log10(results.merged.padj$padj), col=cols, panel.first=grid(),
     main="Volcano plot", xlab="Effect size: log2(fold-change)", ylab="-log10(adjusted p-value)",
     pch=20, cex=0.6)
abline(v=0)
abline(v=c(-1,1), col="red")
abline(h=-log10(alpha), col="red")
dev.off()

plot(results.merged.padj$log2FoldChange, -log10(results.merged.padj$padj), col=cols, panel.first=grid(),
     main="Volcano plot", xlab="Effect size: log2(fold-change)", ylab="-log10(adjusted p-value)",
     pch=20, cex=0.6)
abline(v=0)
abline(v=c(-1,1), col="red") #this makes a vertical line equivalent to a fold change of 2, you can change this if you'd like
abline(h=-log10(alpha), col="red") #this makes a horizontal line equivalent to the alpha that we set (p-adj <= 0.001)
dev.off()

This helps to show that there are roughly an equivalent number of genes that are upregulated and downregulated in the two groups, but there is one gene that is dramatically downregulated (~256 fold). The redlines also set markers to visualize the number of genes that might be considered as differentially expressed. Here we set a fold change of 2 and an adj pvalue of 0.001. All of the genes represented in the upper right and upper left rectangles are the differentially expressed genes at these conditions. This graph also shows that there are quite a few genes with very low padj values with small fold changes. In fact, it seems that most genes have a padj of below 0.001. You could make a histogram of padj values to make an informed decision of setting a threshold. 

This volcano plaot can also be altered to change the color of the significant genes, or even add the names of the genes if you'd like. Let's add gene symbol of highly significant genes to the volcano plot.

### Now add gene names of highly significant genes to the volcano plot.
We will use a log2 fold change of 2 and padj <1e-50 as filters to not overcrowd the plot with gene names.

In [ ]:
#select genes with absolute value of LFC >=2 and padj <= 1e-50
#this will be used to add the names of just these genes
gn.selected <- abs(results.merged.padj$log2FoldChange) >= 2 & results.merged.padj$padj <= 1e-50 

#same code as above to generate volcano plot
alpha <- 0.001 # Threshold on the adjusted p-value
cols <- densCols(results.merged.padj$log2FoldChange, -log10(results.merged.padj$padj))

tiff(file.path(date, outdir, "volcano_with_names.tiff"), res = 100)
plot(results.merged.padj$log2FoldChange, -log10(results.merged.padj$padj), col=cols, panel.first=grid(),
     main="Volcano plot", xlab="Effect size: log2(fold-change)", ylab="-log10(adjusted p-value)",
     pch=20, cex=0.6)
abline(v=0)
abline(v=c(-1,1), col="red")
abline(h=-log10(alpha), col="red")

#add names to volcano plot
text(results.merged.padj$log2FoldChange[gn.selected],
     -log10(results.merged.padj$padj)[gn.selected],
     lab=results.merged.padj$external_gene_name[gn.selected], cex=0.4)

dev.off()

plot(results.merged.padj$log2FoldChange, -log10(results.merged.padj$padj), col=cols, panel.first=grid(),
     main="Volcano plot", xlab="Effect size: log2(fold-change)", ylab="-log10(adjusted p-value)",
     pch=20, cex=0.6)
abline(v=0)
abline(v=c(-1,1), col="red") #this makes a vertical line equivalent to a fold change of 2, you can change this if you'd like
abline(h=-log10(alpha), col="red") #this makes a horizontal line equivalent to the alpha that we set (p-adj <= 0.001)
#dev.off()

#add names to volcano plot
text(results.merged.padj$log2FoldChange[gn.selected],
     -log10(results.merged.padj$padj)[gn.selected],
     lab=results.merged.padj$external_gene_name[gn.selected], cex=0.4)

dev.off()

**note** Now we can easily see that ERBB2 has the smallest padj and CPB1 is the gene that has the largest fold change.

## 2d. Visualizing sample to sample variability from counts data

### Transformation of raw counts for visualization
To calculate differential expressed genes, it is critical to use raw count data. However, for downstream analysis and visualization it is often advantageous to transform the data. We could use our log2 TPM data to generate the PCA plot, but instead I would like to introduce you to built in DESeq2 functions to create an accurate PCA plot from counts. 

There are a couple of methods that we can use to normalize counts to do this. Variance stabilizing transformations (VST) (Tibshirani 1988; Huber et al. 2003; Anders and Huber 2010) produce transformed data on the log2 scale which has been normalized with respect to library size and thus has a slight advantage. The point of VST is to remove the dependence of the variance on the mean, particularly the high variance of the logarithm of count data when the mean is low. 

DESeq2 contains a VST function. We will perform a blinded VST on the dds to generate QC plots that test sample variance.


In [ ]:
#transform count data (variance stabilizing transformation) to aid in visualization
vsd <- vst(dds)

#make vsd into a dataframe and save
vsd.df <- as.data.frame(assay(vsd))
write.csv(vsd.df, file = file.path(date, outdir, "vst.txt"))

### Create PCA plot with VST values
PCA plots show the samples in the 2D plane spanned by their first two principal components. This type of plot is useful for visualizing the overall effect of experimental covariates and batch effects (text from DESeq manual).

In [ ]:
#create PCA plot with variance stabilizing transformation values 
#this code saves the image to file
tiff(file.path(date, outdir, "PCA_vst.tiff"))
pcaData <- plotPCA(vsd, returnData = TRUE, intgroup="BC_subtype")
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=BC_subtype)) +
  geom_point(size=2) +
  # geom_text(aes(label=name),hjust=0.3, vjust=0.1) + #use this line if you want to display sample names over points
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
  coord_fixed()
dev.off()

#this code block prints the PCA block to screen
ggplot(pcaData, aes(PC1, PC2, color=BC_subtype)) +
  geom_point(size=2) +
  # geom_text(aes(label=name),hjust=0.3, vjust=0.1) + #use this line if you want to display sample names over points
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) + 
  coord_fixed()
dev.off()



**note** For the most part there is a clear separation between HER2+ and TNBC samples. This demonstrates that the subtypes are very different, in fact they almost look like completely different diseases. However, there are a few HER2 samples that look similar to TNBC and a few TNBC that look like HER2. This could indicate that the clinical data was wrong or the diagnosis was wrong. Perhaps RNAseq values from 1000s of genes is more sensitive for diagnosis than a single FISH or IHC result of HER2. However, this result could be biological. There could be some HER2 samples that behave as TNBC. If this is the case, the patients with those tumors might not respond to targeted HER2 therapy. Likewise, perhaps some TNBC tumors behave like HER2 samples.

# 3. Generate Heatmaps using top differentially expressed genes
Finally, to bring everything together, we will generate heatmaps with the top differentially expressed genes with different filtering levels based on expression, fold change, and significance. 
There are multiple tools to generate heat maps. In this tutorial we will use [heatmap.3] (https://www.rdocumentation.org/packages/GMD/versions/0.3.3/topics/heatmap.3), which is a varient of gplots heatmap.2. I suggest reading the manual to understand all options. We will create the heatmap from  the more traditional abundance (TPM) data that we already downloaded and processed but a heatmap could be made from the vst data as well. 

First install heatmap.3.

In [ ]:
#install heatmap.3 from source
#the output will give you an SHA-1 hash of the file
source_url("https://raw.githubusercontent.com/obigriffith/biostar-tutorials/master/Heatmaps/heatmap.3.R")

## Filter significant DEGs
Through DESeq, we identified the differentially expressed genes. However, it is customary to narrow this list by applying a few filters based on:
1) Expression
2) Fold Change
3) Significance

We first will filter by expression because genes expressed at low levels are the noisiest in RNAseq and the most difficult to validate through wet-lab experiments. In my hands, genes with a TPM of 1 are readily validated. Typically, I use 0.5-1TPM as my cutoff, and typically will filter if expressed below that in >90% of samples. While the cutoffs will largely depend on your choices and dataset, for this exercise, we will filter genes that are expressed less than TPM of 1 at the 90th percentile in either TNBC or HER2 positive. 

Fold change filters are typically set at 1.5 or 2.0

Significane thresholds are typically at least <0.01 padj...but in a large dataset like this, smaller thresholds are common.

In [ ]:
#Filter by expression. To do this we will use the tpm values that we downloaded and collapsed to gene level earlier

#filter based on minimum 90th percentile expression of tpm of 1 in HER2 or TNBC samples
exp.cutoff <- 1
results.sig.exp <- results.merged.padj[results.merged.padj$ninetieth_percentile_her2 >= exp.cutoff | results.merged.padj$ninetieth_percentile_tnbc >= exp.cutoff, ]

#filter based on FC of greater than 1.5 or less than -1.5
fc.cutoff <- 1.5
results.sig.exp.fc <- results.sig.exp[(2^results.sig.exp$log2FoldChange >= fc.cutoff) | (2^results.sig.exp$log2FoldChange <= 1/fc.cutoff), ]

#filter with a padj of 0.001
padj.cutoff <- 0.001
results.sig.exp.fc.padj <- results.sig.exp.fc[results.sig.exp.fc$padj <= padj.cutoff, ]

#Determine the number of significant genes that pass the filters
numSigGenes <- nrow(results.sig.exp.fc.padj)
message("Number of significant genes")
numSigGenes

#Number of significantly upregulated genes
message("Number of significantly upregulated genes")
nrow(results.sig.exp.fc.padj[(2^results.sig.exp$log2FoldChange >= fc.cutoff), ])


#Number of significantly downregulated genes
message("Number of significantly downregulated genes")
nrow(results.sig.exp.fc.padj[(2^results.sig.exp$log2FoldChange <= 1/fc.cutoff), ])

#determine the names of the genes
filtered.genes <- row.names(results.sig.exp.fc.padj)

#save the table of significant genes that can then be imported into a pathway analysis software or explored in greater depth 
write.csv(results.sig.exp.fc.padj, file = file.path(date, outdir, "her2vstnbc_0.001p_fc1.5_exp90_1.csv"))




Note that the number of significant genes is less than the original summary of the output at the end of Notebook 3. That summary stated 3996 genes were significantly upregulated and 3375 genes were signfinicantly downregulated. At the end of this notebook, you will adjust the filtering levels to create a 'stringent' table of DE genes.

## Create the heatmap
See comments for rationale behind the code

In [ ]:
#To visualize our DESeq and filtering results, we need to create a matrix (not a dataframe) 
#of abundance (tpm) of all samples (HER2+/TNBC her2.ids.final and tnbc.ids.final) 
#and all genes that are significantly different and pass filtering criteria (filtered.genes)
#remember her2.tpm.df is unlogged tpm data we generated above from all her2 samples
#the same is true of tnbc.tpm.df

hm.her2.tpm.filtered <- her2.tpm.df[filtered.genes, ]
hm.tnbc.tpm.filtered <- tnbc.tpm.df[filtered.genes, ]
hm.tpm.filtered.combined <- cbind(hm.her2.tpm.filtered, hm.tnbc.tpm.filtered)

#for visualization purposes, tpm data is often better log2 transformed
#note we need to add a small value to each tpm so that we do not try to log 0
hm.log.tpm <- log2(hm.tpm.filtered.combined + 0.001)

#likewise median centering the rows allows for easier visualization and interpration

#rowMedians calculates the median of each row. we then subtract the median from every value
row.medians <- rowMedians(as.matrix(hm.log.tpm))
hm.log.median.centered.tpm <- hm.log.tpm - row.medians

#check all of the transformations
message("untransformed")
head(hm.tpm.filtered.combined)

message("log2 transformed")
head(hm.log.tpm)

message("median centered rows")
head(hm.log.median.centered.tpm)
row.medians[1:6]


In [ ]:
#we need to establish colors for her2 and tnbc to label the columns (samples) as we have 
#too many samples to read the names
#in this command we establish that green represents her2 and red tnbc
#we also establish how many her2 and tnbc samples there are using the rep (repeat) command
colcolors <- cbind(subtype = c(rep("green", ncol(hm.her2.tpm.filtered)), rep("red", ncol(hm.tnbc.tpm.filtered))))

#next we need to establish what the colors in the heatmap represent and how many colors to use
#breaks establishes how many colors. in this case we go from -2 to 2 by 0.05.
#-2 and 2 were selected as they will show all samples greater than 4 fold from the median as dark colors
#samples closer to the median will have a duller color
#these numbers can be altered for best visualization purposes
breaks <- seq(-2, 2, by = 0.05)

#this command establishes what colors to use. In this example values around the median will be black
#higher than the median yellow, and lower than the median will be blue.
#the colorpanel command helps break all of the colors into different gradients based on the number of breaks
#note that you can change the colors
#also note that there are only so many different versions of each color
col<- colorpanel(n = length(breaks) - 1, low = "blue", mid = "black", high = "yellow")


#heatmap of filtered genes clustering both by gene and sample
#tpm data has to be a matrix
#ColSideColors establishes color for each column as defined in colcolors
#breaks and col are described and defined above
#note that because we have so many genes and samples the labels cannot be read, so we removed them with the labRow, LabCol options.
#dendogram tells heatmap to do clustering on rows, columns, both, or neither
#density info places a histogram in the legend to see the frequency of genes at each level
heatmap.3(as.matrix(hm.log.median.centered.tpm), ColSideColors = colcolors, breaks = breaks, col = col, dendrogram = "both", density.info = "histogram", labRow = FALSE, labCol = FALSE)
legend(x=0.96, y=0.84, xpd = TRUE, title = "Subtype", legend=c("HER2","TNBC"), fill=c("green","red"), border=FALSE, bty="n", cex=0.6, y.intersp = 0.7, inset = 0.1)
dev.off()


#code below is the same but saves the heatmap to file
tiff(file.path(date, outdir, "DEGs_heatmap_0.001padj_1.5FC_1tpm_90thpercentile.tiff"))
heatmap.3(as.matrix(hm.log.median.centered.tpm), ColSideColors = colcolors, breaks = breaks, col = col, dendrogram = "both", density.info = "histogram", labRow = FALSE, labCol = FALSE)
legend(x=0.96, y=0.84, xpd = TRUE, title = "Subtype", legend=c("HER2","TNBC"), fill=c("green","red"), border=FALSE, bty="n", cex=0.6, y.intersp = 0.7, inset = 0.1)
dev.off()



The heatmap demonstrates that with the filtered list of differentially expressed genes, the samples largely cluster based on breast cancer subtype. However, there are still a few outliers. This is a similar result to the PCA plot of the counts data. The heatmap also demonstrates that there is roughly the same number of up and down-regulated genes in the filtered set.

## What can we do with the results?
The answer to this goes beyond the goals of this notebook. However, it is quite common to see if the DEGs are enriched in any biological or clinically actionable pathways. This can be done through Gene Set Enrichment Analysis (GSEA) or with other pathway analysis tools such as Ingenuity Pathway Analysis. Another option is to use the DEGs as features in a causal discovery or some other machine learning problem. An example of a nice causal discovery tool is [Tetrad](http://www.phil.cmu.edu/tetrad/) developed out of Pitt and CMU in the Center for Causal Discovery. There is even another github exercise at the DBMI that is in development on how to perform caucal analysis with Tetrad.

___
___
___

## In the code box below change the expression, fold change, and padj cutoffs to determine which cutoff used above is the most stringent

1. What are the number of significant genes if fold change cutoff is 2.0, but other paramaters above are the same?
2. What are the number of significant genes if padj.cutoff is set to 0.00000000001, but other paramaters above are the same?
3. What are the number of significant genes if exp.cutoff is 1 for a MINIMUM instead of median in either TNBC or HER2? *note min is a function in R.

create a final dataframe named results.stringent with all 3 of these thresholds.

## Now you make a heatmap of results.stringent
compare and contrast the two heatmaps.